In [62]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [63]:
%%R
library(readxl)
library(dplyr)

In [64]:
%%R
df_ipl=read.csv("IPL_ball_by_ball_updated till 2024.csv")
salary =read_excel("IPL SALARIES 2024.xlsx")

In [66]:
%%R
head(df_ipl)

  Match.id       Date  Season          Batting.team                Bowling.team
1   335982 18-04-2008 2007/08 Kolkata Knight Riders Royal Challengers Bangalore
2   335982 18-04-2008 2007/08 Kolkata Knight Riders Royal Challengers Bangalore
3   335982 18-04-2008 2007/08 Kolkata Knight Riders Royal Challengers Bangalore
4   335982 18-04-2008 2007/08 Kolkata Knight Riders Royal Challengers Bangalore
5   335982 18-04-2008 2007/08 Kolkata Knight Riders Royal Challengers Bangalore
6   335982 18-04-2008 2007/08 Kolkata Knight Riders Royal Challengers Bangalore
  Innings.No Ball.No  Bowler     Striker Non.Striker runs_scored extras
1          1     0.1 P Kumar  SC Ganguly BB McCullum           0      1
2          1     0.2 P Kumar BB McCullum  SC Ganguly           0      0
3          1     0.2 P Kumar BB McCullum  SC Ganguly           0      1
4          1     0.3 P Kumar BB McCullum  SC Ganguly           0      0
5          1     0.4 P Kumar BB McCullum  SC Ganguly           0      0
6       

In [67]:
%%R
colnames(df_ipl)

 [1] "Match.id"            "Date"                "Season"             
 [4] "Batting.team"        "Bowling.team"        "Innings.No"         
 [7] "Ball.No"             "Bowler"              "Striker"            
[10] "Non.Striker"         "runs_scored"         "extras"             
[13] "type.of.extras"      "score"               "score.wicket"       
[16] "wicket_confirmation" "wicket_type"         "fielders_involved"  
[19] "Player.Out"         


In [69]:
%%R
# Grouping and Summarizing the Data
grouped_data <- df_ipl %>%
  group_by(Season, Innings.No, Striker, Bowler) %>%
  summarise(
    runs_scored = sum(runs_scored, na.rm = TRUE),
    wicket_confirmation = sum(wicket_confirmation, na.rm = TRUE)
  ) %>%
  ungroup()

`summarise()` has grouped output by 'Season', 'Innings.No', 'Striker'. You can
override using the `.groups` argument.


In [71]:
%%R
grouped_data

# A tibble: 48,786 × 6
   Season  Innings.No Striker  Bowler        runs_scored wicket_confirmation
   <chr>        <int> <chr>    <chr>               <int>               <int>
 1 2007/08          1 A Chopra DP Vijaykumar           1                   0
 2 2007/08          1 A Chopra DW Steyn                1                   1
 3 2007/08          1 A Chopra GD McGrath              2                   0
 4 2007/08          1 A Chopra PJ Sangwan              6                   1
 5 2007/08          1 A Chopra RP Singh                9                   0
 6 2007/08          1 A Chopra SB Bangar               9                   0
 7 2007/08          1 A Chopra SB Styris               1                   0
 8 2007/08          1 A Chopra SM Pollock              1                   0
 9 2007/08          1 A Chopra VY Mahesh               5                   0
10 2007/08          1 A Chopra WPUJC Vaas              1                   0
# ℹ 48,776 more rows
# ℹ Use `print(n = ...)` to see 

In [72]:
%%R
# Calculate total runs each year
total_runs_each_year <- grouped_data %>%
  group_by(Season, Striker) %>%
  summarise(total_runs = sum(runs_scored, na.rm = TRUE)) %>%
  ungroup()

#calculate total wickets each year
total_wicket_each_year <- grouped_data %>%
  group_by(Season, Bowler) %>%
  summarise(total_wickets = sum(wicket_confirmation, na.rm = TRUE)) %>%
  ungroup()

`summarise()` has grouped output by 'Season'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'Season'. You can override using the
`.groups` argument.


In [73]:
%%R
total_runs_each_year

# A tibble: 2,598 × 3
   Season  Striker        total_runs
   <chr>   <chr>               <int>
 1 2007/08 A Chopra               42
 2 2007/08 A Kumble               13
 3 2007/08 A Mishra               37
 4 2007/08 A Mukund                0
 5 2007/08 A Nehra                 3
 6 2007/08 A Symonds             161
 7 2007/08 AA Noffke               9
 8 2007/08 AB Agarkar             54
 9 2007/08 AB Dinda                2
10 2007/08 AB de Villiers         95
# ℹ 2,588 more rows
# ℹ Use `print(n = ...)` to see more rows


In [30]:
%%R
total_wicket_each_year

# A tibble: 1,929 × 3
   Season  Bowler         total_wickets
   <chr>   <chr>                  <int>
 1 2007/08 A Kumble                   8
 2 2007/08 A Mishra                  11
 3 2007/08 A Nehra                   14
 4 2007/08 A Nel                      1
 5 2007/08 A Symonds                  0
 6 2007/08 AA Noffke                  1
 7 2007/08 AB Agarkar                 8
 8 2007/08 AB Dinda                  11
 9 2007/08 AD Mascarenhas             2
10 2007/08 AM Nayar                   1
# ℹ 1,919 more rows
# ℹ Use `print(n = ...)` to see more rows


In [74]:
%%R
install.packages("stringdist")

# Function to match names using stringdist package
match_names <- function(name, names_list) {
  dist <- stringdist::stringdist(name, names_list, method = "jw")
  match <- names_list[which.min(dist)]
  score <- 1 - min(dist)
  return(ifelse(score >= 0.8, match, NA)) # Use a threshold score of 0.8
}

# Apply fuzzy matching to match names
salary$Matched_Player <- sapply(salary$Player, match_names, names_list = total_runs_each_year$Striker)

# Merge the DataFrames on the matched names
df_merged <- merge(salary, total_runs_each_year, by.x = "Matched_Player", by.y = "Striker")


(as ‘lib’ is unspecified)







	‘/tmp/RtmpNhBswc/downloaded_packages’



In [75]:
%%R
# Subset data for the last three years
df_merged <- df_merged %>%
  filter(Season %in% c("2021", "2022", "2023"))

# Check the unique seasons
unique(df_merged$Season)


[1] "2023" "2022" "2021"


In [76]:
%%R
print(colnames(df_merged))

[1] "Matched_Player" "Player"         "Salary"         "Rs"            
[5] "international"  "iconic"         "Season"         "total_runs"    


In [77]:
%%R
# Independent and dependent variables
X <- df_merged$total_runs
y <- df_merged$Rs

# Split the data into training and test sets (80% for training, 20% for testing)
set.seed(42)
train_indices <- sample(seq_len(nrow(df_merged)), size = 0.8 * nrow(df_merged))
train_data <- df_merged[train_indices, ]
test_data <- df_merged[-train_indices, ]

# OLS regression model
model <- lm(Rs ~ total_runs, data = train_data)

# Summary of the regression model
summary(model)

# Using the broom package to get tidy output
tidy_model <- broom::tidy(model)
print(tidy_model)

# Predicting on the test set
predictions <- predict(model, newdata = test_data)

# Calculate R-squared and Mean Absolute Percentage Error
r_squared <- summary(model)$r.squared
mape <- mean(abs((test_data$Rs - predictions) / test_data$Rs)) * 100

# Print metrics
cat("R-squared:", r_squared, "\n")
cat("MAPE:", mape, "\n")


# A tibble: 2 × 5
  term        estimate std.error statistic       p.value
  <chr>          <dbl>     <dbl>     <dbl>         <dbl>
1 (Intercept)   335.      53.4        6.28 0.00000000700
2 total_runs      1.08     0.234      4.63 0.0000103    
R-squared: 0.1642022 
MAPE: 429.0071 


In [60]:
%%R
# Function to match names using stringdist package
match_names <- function(name, names_list) {
  dist <- stringdist::stringdist(name, names_list, method = "jw")
  match <- names_list[which.min(dist)]
  score <- 1 - min(dist)
  return(ifelse(score >= 0.8, match, NA)) # Use a threshold score of 0.8
}

# Apply fuzzy matching to match names for wicket
salary$Matched_Player <- sapply(salary$Player, match_names, names_list =total_wickets$Bowler)

# Merge the DataFrames on the matched names
df_merged <- merge(salary,total_wicket_each_year, by.x = "Matched_Player", by.y = "Bowler")


  object 'total_wickets' not found




Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  object 'total_wickets' not found


In [78]:
%%R
# Independent and dependent variables in wickets
X <- df_merged$total_wickets
y <- df_merged$Rs

# Split the data into training and test sets (80% for training, 20% for testing)
set.seed(42)
train_indices <- sample(seq_len(nrow(df_merged)), size = 0.8 * nrow(df_merged))
train_data <- df_merged[train_indices, ]
test_data <- df_merged[-train_indices, ]

# OLS regression model
model <- lm(Rs ~ total_wickets, data = train_data)

# Summary of the regression model
summary(model)

# Using the broom package to get tidy output
tidy_model <- broom::tidy(model)
print(tidy_model)

# Predicting on the test set
predictions <- predict(model, newdata = test_data)

# Calculate R-squared and Mean Absolute Percentage Error
r_squared <- summary(model)$r.squared
mape <- mean(abs((test_data$Rs - predictions) / test_data$Rs)) * 100

# Print metrics
cat("R-squared:", r_squared, "\n")
cat("MAPE:", mape, "\n")


Error in eval(predvars, data, env) : object 'total_wickets' not found


In [78]:
#OLS for predicting salary of bowlers
